In [ ]:
#default_exp relationships

In [ ]:
#no_test
#dependencies

#nlp packages
import spacy
from spacy.util import minibatch, compounding

#manipulation of tables/arrays
import pandas as pd
import numpy as np

#internal imports
from ssda_nlp.collate import *
from ssda_nlp.split_data import *
from ssda_nlp.modeling import *
from ssda_nlp.model_performance_utils import *
from ssda_nlp.xml_parser import *
from ssda_nlp.unstructured2markup import *

In [ ]:
#export

def id_unique_individuals(entry_text, entities, record_type):
    '''
    identifies all unique individuals that appear in an entry (i.e. removing all multiple mentions of the same person)
        entry_text: the full text of a single entry, ported directly from spaCy to ensure congruity
        entities: entities of all kinds extracted from that entry by an NER model
        record_type: simple flag indicating whether records are baptisms, marriages, burials, etc. (this can also be determined
        programmatically and may be deprecated later)
        
        returns: a list of the unique individuals who appear in an entry
    '''
    
    people_df = entities.loc[entities['pred_label'] == 'PER']
    people_df.reset_index(inplace=True)
    people_df = people_df.drop('index',axis=1)
    unique_individuals = people_df['pred_entity'].unique()
    
    return unique_individuals

### find_sus is broken

In [ ]:
#export

def find_sus(entry_text, entities, sus_df, index):
    '''
    identifies corner cases: all entries where there are multiple entities that 1) have the same first name appearing 
        multiple times, 2) have compound names and then a segment of that name appearing, and 3) have a full name with 
        the first name by itself appearing
    Note that this should not be used in tandem with id_unique_individuals, as that function just drops the duplicate names
    
    params:
        entry_text: actual text for comparison
        entities: df of entities identified
        sus_df: either the empty df body or the df from previously loop iterations
        i: current row that the loop is on in DEMO_DF
    
    returns: df of all the entries that may be corner cases, in the same form demo_df, but with two added id columns
    '''
    #Set up
    people_df = entities.loc[entities['pred_label'] == 'PER']
    people_df.reset_index(inplace=True)
    people_df = people_df.drop('index',axis=1)
    
    my_rows = len(people_df.index)
    hold = my_rows * [0]
    people_df['name_status'] = hold
    first_names = []
    check_against = []
    dups = 0
    sus = 0
    
    #Get a list of all the first names that appear in the entities/people_df
    #  This is definitely not the most computationally efficient way to do this
    for i in range(my_rows):
        #Separate people based on whether it is a first name or a full/compound name
        if (" " in people_df.iloc[i,1]) or ("-" in people_df.iloc[i,1]):
            check_against.append(people_df.iloc[i,1])
        elif ~(" " in people_df.iloc[i,1]): #No spaces thus we are assuming it is a first name
            first_names.append(people_df.iloc[i,1])
    #Check to see whether they are subsets of full/compound names
    if len(first_names)>0 and len(check_against)>0:
        for j in range(len(first_names)):
            for k in range(len(check_against)):
                if first_names[j] in check_against[k]:
                    #Mark this entire entry as sus
                    sus = 1
    #Generally check to see if there are any duplicate entities (same name) in the entry
    if people_df['pred_entity'].duplicated().any():
        dups = 1;
    #Set the status column
    if sus and dups:
        status = 11 #ie both sus and dups are true
    elif sus:
        status = 10 #ie sus true, dups false
    elif dups:
        status = 0.01 #ie sus false, dups true
    else:
        status = 0
    #ie if the entry is suspect or has duplicates, then add it to sus_df
    if status>0:
        if len(sus_df.index)<1:
            data = [{'vol_titl':demo_df.iloc[index,0], 'vol_id':demo_df.iloc[index,1], 'fol_id':demo_df.iloc[index,2],
                    'text':demo_df.iloc[index,3],'entry_no':entry_no,'suspect':status}]
            sus_df = pd.DataFrame(data)
        else:
            sus_df = sus_df.append({'vol_titl':demo_df.iloc[index,0], 'vol_id':demo_df.iloc[index,1], 'fol_id':demo_df.iloc[index,2],
                    'text':demo_df.iloc[index,3],'entry_no':entry_no,'suspect':status},ignore_index=True)
    return sus_df

### split_name_col is not completed yet

In [ ]:
#export

def split_name_col(people_df):
    '''
    from the fed in entities, strips DF to only include people, then separates based on if it is a first name or a full name
    
    '''
    #Set up
    my_rows = len(people_df.index)
    hold = my_rows * [0]
    people_df['name_status'] = hold
    
    #Separate into two based on first/single and full name status
    for i in range(my_rows):
        if "-" in people_df.iloc[i,1]:
            people_df.iloc[i,5] = 2 #2 therefore represents compound name
        elif " " in people_df.iloc[i,1]:
            people_df.iloc[i,5] = 1 #1 therefore represents a full name
        else: #Must be a single name
            #0 therefore represents a full name
            pass
    first_n = people_df[people_df.name_status == 0]
    full_n = people_df[people_df.name_status == 1]
    cmpd_n = people_df[people_df.name_status == 2]
    
    print("DF of first names")
    display(first_n.head())
    print("DF of full names")
    display(full_n.head())
    print("DF of compound names")
    display(cmpd_n.head())
    print("---------------------")
    
    return first_n, full_n, cmpd_n

### disambiguate
1. Doesn't do anything once entities are separated

In [ ]:
#export

def disambiguate():
    '''
    goes through the problem cases previously identified and then applies split_name_col to break the entities down into
        the ones that may be 
    '''
    people_df = entities.loc[entities['pred_label'] == 'PER']
    people_df.reset_index(inplace=True)
    people_df = people_df.drop('index',axis=1)
    
    first_n, full_n, cmpd_n = split_name_col(people_df)
    

In [ ]:
#export

def determine_principals(entry_text, entities, n_principals):
    '''
    determines the principal of a single-principal event
        entry_text: the full text of a single entry, ported directly from spaCy to ensure congruity
        entities: entities of all kinds extracted from that entry by an NER model
        n_principals: expected number of principals
        
        returns: the principal(s) of the event in question, or None if no principal can be identified
    '''
    
    entry_text = entry_text.lower()
    
    if n_principals == 1:        
        principals = None
        
        for index, entity in entities.iterrows():
            if entity['pred_label'] == 'PER' and entity['pred_start'] <= 20:
                principals = entity['pred_entity']
                
        if principals == None:            
            prox = entry_text.find('oleos')
            if prox != -1:
                for index, entity in entities.iterrows():
                    if entity['pred_label'] == 'PER' and (abs(entity['pred_start'] - prox) <= 10):
                        principals = entity['pred_entity']
                        
        if principals == None:
            prox = entry_text.find('nombre')
            if prox != -1:
                for index, entity in entities.iterrows():
                    if entity['pred_label'] == 'PER' and (abs(entity['pred_start'] - prox) <= 10):
                        principals = entity['pred_entity']                        
        
    elif n_principals == 2:
        print("That number of principals is not supported yet.")
        return None
        #process marriage principals
    else:
        print("Invalid number of principals.")
        return None
    
    return principals

In [ ]:
#export

def determine_event_date(entry_text, entities, event_type, volume_metadata):
    '''
    determines the date of a specific event
        entry_text: the full text of a single entry, ported directly from spaCy to ensure congruity
        entities: entities of all kinds extracted from that entry by an NER model
        event_type: this could be either a valid record_type OR a secondary event like a birth
        volume_metadata: metadata for the volume that the entry comes from, built by retrieve_volume_metadata
        
        returns: the date of the event in question, or None if no date can be identified
    '''
    date = None
    
    if event_type != volume_metadata["type"]:        
        primary_event_date = determine_event_date(entry_text, entities, event_type, volume_metadata)
        for index, entity in entities.iterrows():
            if entity['pred_label'] == 'DATE' and entity['pred_entity'] != primary_event_date:
                date = entity['pred_entity']
    
    elif volume_metadata["type"] == "baptism":
        entry_length = len(entry_text)
        
        for index, entity in entities.iterrows():
            if entity['pred_label'] == 'DATE' and entity['pred_start'] <= (entry_length / 3):
                date = entity['pred_entity']        
                
    else:
        date = "That event type is not supported yet."
        
    return date

In [ ]:
#export

def determine_event_location(entry_text, entities, event_type, volume_metadata):
    '''
    determines the location of a specific event
        entry_text: the full text of a single entry, ported directly from spaCy to ensure congruity
        entities: entities of all kinds extracted from that entry by an NER model
        event_type: this could be either a valid record_type OR a secondary event like a birth
        volume_metadata: metadata for the volume that the entry comes from, built by retrieve_volume_metadata
        
        returns: the location of the event in question, or None if no date can be identified
    '''
    location = None
    
    if event_type == volume_metadata["type"]:
        location = volume_metadata["institution"]    
    else:
        location = "That event type is not supported yet."
    
    return location

In [ ]:
#export

def identify_cleric(entry_text, entities):
    '''
    identifies the cleric(s) associated with a sacramental entry
        entry_text: the full text of a single entry, ported directly from spaCy to ensure congruity
        entities: entities of all kinds extracted from that entry by an NER model        
        
        returns: the associated cleric(s), or None if no date can be identified
    '''
    clerics = None
    
    for index, entity in entities.iterrows():
            if ((entity['pred_label'] == 'PER') and ((len(entry_text) - entity['pred_end']) <= 10) and (len(entry_text) > 100)):
                clerics = entity['pred_entity']
            #going to keep this condition for now, but it can create false positives when long, incorrect entities are extracted
            #from short and/or garbled entries
            elif (entity['pred_entity'] != None) and (len(entry_text) - entity['pred_end'] <= 2) and (entity['pred_label'] == 'PER'):
                clerics = entity['pred_entity']                                                 
                
    if clerics == None:
        pvs_label = None
        pvs_end = None
        for index, entity in entities.iterrows():
            if entity['pred_label'] == 'PER' and pvs_label == 'DATE' and (entity['pred_start'] - pvs_end) <= 15:
                clerics = entity['pred_entity']                
            pvs_label = entity['pred_label']
            pvs_end = entity['pred_end']
    
    if clerics == None:
        entry_text = entry_text.lower()
        for index, entity in entities.iterrows():
            if entity['pred_label'] == 'PER' and entry_text.find("cura", entity['pred_start'] + len(entity['pred_entity'])) != -1 and ((entry_text.find("cura", entity['pred_start'] + len(entity['pred_entity']))) - entity['pred_end']) <= 15:
                clerics = entity['pred_entity']                
    
    return clerics

In [ ]:
#export

def build_event(entry_text, entities, event_type, principals):
    '''
    builds out relationships related to a baptism or burial event
        entry_text: the full text of a single entry, ported directly from spaCy to ensure congruity
        entities: entities of all kinds extracted from that entry by an NER model
        event_type: this could be either a valid record_type OR a secondary event like a birth
        
        returns: structured representation of these relationships, including (but not necessarily limited to)
        the event's principal, the date of the event, the location of the event, and the associated cleric
    '''   
    
    date = determine_event_date(entry_text, entities, event_type)
    location = determine_event_location(entry_text, entities, event_type)
    cleric = identify_cleric(entry_text, entities)
    
    return event_relationships

In [ ]:
#export

def build_relationships(entry_text, cpd_identifier, entities, path_to_volume_xml):
    '''
    Master function that will combine all helper functions built above
        entry_text: the full text of a single entry, ported directly from spaCy to ensure congruity
        cpd_identifier: compound id containing volume id, image id, and entry id
        entities: entities of all kinds extracted from that entry by an NER model
        record_type: simple flag indicating whether records are baptisms, marriages, burials, etc. (this can also be determined
        programmatically and may be deprecated later)
            
        returns: structured data (specific format to be named later) containing all relationships extracted
    '''
    #unique_individuals = id_unique_indviduals(entry_text, entities, record_type)
    
    volume_metadata = retrieve_metadata(path_to_volume_xml)
    
    if volume_metadata["type"] == "baptism":
        principals = determine_principals(entry_text, entities, 1)
        #event_relationships = build_event(entry_text, entities, "baptism", principals)               
        #interpersonal_relationships = process_interpersonal(entry_text, entities)
        #characteristics = process_characteristics(entry_text, entities, interpersonal_relationships)        
    elif volume_metadata["type"] == "marriage":        
        #process marriage record
        print("That record type is not supported yet.")
        return None
    elif volume_metadata["type"] == "burial":
        #process burial record
        print("That record type is not supported yet.")
        return None
    else:
        print("That record type is not supported yet.")
        return None
    
    #code that turns pieces defined above into well-formed relationships
    
    return relationships

## Unit testing

### Load trained model and entry data

In [ ]:
#no_test

trained_model = load_model('models/st_aug_bapt_2', language="es", verbose='True')

Loaded model 'models/st_aug_bapt_2'


In [ ]:
#no_test

path_to_transcription = "transcriptions\\239746.xml"
demo_df = parse_xml_v2(path_to_transcription)
demo_df.head()

,vol_titl,vol_id,fol_id,text,entry_no
0,Baptisms - 1793-1807,239746,1013,"1. María Dolores Sanchez Lunes, día veinte y ...",1013-1
1,Baptisms - 1793-1807,239746,1013,"2. Antonio Guillo Miercoles, día veinte de No...",1013-2
2,Baptisms - 1793-1807,239746,1014,"3. María Juana Francisca Fish Domingo, día ve...",1014-1
3,Baptisms - 1793-1807,239746,1014,"4. Maria Teresa Camel Domingo, día veinte y q...",1014-2
4,Baptisms - 1793-1807,239746,1015,"Maria Josefa Andrea de la Puente Miércoles, d...",1015-1


In [ ]:
#no_test

volume_metadata = retrieve_volume_metadata(path_to_transcription)
print(volume_metadata)

{'type': 'baptism', 'country': 'United States', 'state': 'Florida', 'city': 'St. Augustine', 'institution': 'Archives of the Diocese of St. Augustine', 'id': 'St. Augustine', 'title': 'Baptisms - 1793-1807'}


### Apply model to entry data

In [ ]:
#no_test

ent_preds_df, metrics_df, per_ent_metrics = test_model(trained_model, demo_df, "entry_no", "text", score_model=False)
ent_preds_df.head(20)

,entry_no,pred_entity,pred_label,pred_start,pred_end
0,1013-1,María Dolores Sanchez,PER,3,24
1,1013-1,Lunes,CHAR,26,31
2,1013-1,Don Miguel o’Reilly,PER,99,118
3,1013-1,Teniente de Cura Beneficiado,CHAR,119,147
4,1013-1,Vicario,CHAR,149,156
5,1013-1,Juez Eclesiástico Auxiliar,CHAR,160,186
6,1013-1,Igle sia parroquial,LOC,195,214
7,1013-1,provincia de San Agustín,LOC,218,242
8,1013-1,Florida Oriental,LOC,249,265
9,1013-1,niña,CHAR,305,309


### id_unique_individuals

In [ ]:
#no_test

for index, row in demo_df.iterrows():
    entry_no = row['entry_no']
    entry_text = row['text']
    entities = ent_preds_df.loc[ent_preds_df['entry_no'] == entry_no]
    print(id_unique_individuals(entry_text, entities, volume_metadata["type"]))
    if index > 15:
        break

['María Dolores Sanchez' 'Don Miguel o’Reilly' 'Juan' 'Deliby Naturales'
 'Don Francisco Sanchez' 'Maria Dolores' 'José Rivas'
 'María de la Luz Blanco']
['Antonio Guillo' 'Don Miguel o’Reilly' 'Isaac Guillo' 'Sara Ca'
 'Juan Antonio' 'Antonio Pellice' 'Susana Pellicer']
['María Juana Francisca Fish' 'Don Miguel o’Reilly' 'Don Jeremías Fish'
 'Eva Fish' 'Maria Juana Francisca' 'Lorenzo Capó'
 'Juana Joaquina Gonzalez']
['Maria Teresa Camel' 'Don Miguel o’Reilly' 'Andres Camel' 'Eva Fish'
 'Maria Tere sa' 'Francisco Huiten' 'Andrea Ortigas']
['Maria Josefa Andrea de la Puente' 'Don Miguel o’Reilly'
 'Maria Prudencia Zayas' 'Don Armando de la Puente' 'Maria Josefa Andrea'
 'Domingo' 'Doña Corolinda Josefa Leonardy']
['Juana Teresa de Jesus Gernon' 'Don Miguel o’Reilly'
 'Don Guil lermo Gernon' 'Juana Teresa de Jesus' 'Doña Teresa Rodríguez']
['Maria Josepha de los' 'Dolores Guadara ma' 'Don Miguel o’Reilly'
 'Don Mateo Guadarama' 'Maria Josefa de los'
 'Doña Catalina Gerónima Porras']
['

### find_sus

In [ ]:
sus_df = pd.DataFrame(columns=['vol_titl', 'vol_id', 'fol_id', 'text','entry_no','suspect'])

#For reference: suspect codes:
#0.01 means there are duplicates but there aren't first names that appear alone
#10.00 means there are first names that are subsets of full names, but no duplicates
#11 means there are both first names that are subsets and duplicates

for index, row in demo_df.iterrows():
    entry_no = row['entry_no']
    entry_text = row['text']
    entities = ent_preds_df.loc[ent_preds_df['entry_no'] == entry_no]
    sus_df = find_sus(entry_text, entities, sus_df, index)
    if index > 150: #Has 481 rows
        break

print(f"Here is the df of sus entries, with a length of ", len(sus_df.index))
display(sus_df.head(20))

Here is the df of sus entries, with a length of  51


,vol_titl,vol_id,fol_id,text,entry_no,suspect
0,Baptisms - 1793-1807,239746,1019,"Antonio Joseph Dimas Pater Adulto Domingo, dí...",1019-1,0.01
1,Baptisms - 1793-1807,239746,1019,"Maria Dolores Judith Smith Adulta Domingo, dí...",1019-2,0.01
2,Baptisms - 1793-1807,239746,1021,"Mariana Guadalupe Perpal Sabado, día tres de ...",1021-1,10.00
3,Baptisms - 1793-1807,239746,1022,"19. Juan Baptista Isnardy Martes, día veinte ...",1022-1,0.01
4,Baptisms - 1793-1807,239746,1022,"Roberto Macqueen Miércoles Martes, día veinte...",1022-2,11.00
5,Baptisms - 1793-1807,239746,1023,"21. Thomas Macqueen Miércoles, día veinte y un...",1023-1,10.00
6,Baptisms - 1793-1807,239746,1023,"Catarina Macqueen Miércoles, día veinte y uno ...",1023-2,10.00
7,Baptisms - 1793-1807,239746,1024,"Juana Macqueen Miércoles, día veinte y uno de ...",1024-1,10.00
8,Baptisms - 1793-1807,239746,1024,"Benjamin Macqueen Miércoles, día veinte y uno...",1024-2,11.00
9,Baptisms - 1793-1807,239746,1025,"Pablo Macqueen Miércoles, día veinte y uno de...",1025-1,10.00


### disambiguate

In [ ]:
for index, row in sus_df.iterrows():
    entry_no = row['entry_no']
    entry_text = row['text']
    status = row['suspect']
    entities = ent_preds_df.loc[ent_preds_df['entry_no'] == entry_no]
    if status == 0.01:
        pass #There are only duplicates, so might want to deal with this separately
    elif status > 9: #ie sus is true, disambiguation needed
        disambiguate()
    if index > 10: #Has 481 rows
        break

DF of first names


,entry_no,pred_entity,pred_label,pred_start,pred_end,name_status
4,1021-1,Mariana,PER,588,595,0


DF of full names


,entry_no,pred_entity,pred_label,pred_start,pred_end,name_status
0,1021-1,Mariana Guadalupe Perpal,PER,0,24,1
1,1021-1,Don Miguel O’Reilly,PER,91,110,1
2,1021-1,Antonio Perpal,PER,395,409,1
3,1021-1,Anna Overstrod,PER,412,426,1
5,1021-1,Juan Baptista Huiten,PER,617,637,1


DF of compound names


...

DF of full names


,entry_no,pred_entity,pred_label,pred_start,pred_end,name_status
0,1026-1,Thomas Macqueen,PER,4,19,1
1,1026-1,Don Thomas Hafsett,PER,96,114,1
3,1026-1,Dafíny Macqueen,PER,365,380,1
4,1026-1,Don Juan Macqueen,PER,436,453,1
6,1026-1,Doctor Don Thomas Sterling,PER,580,606,1


DF of compound names


,entry_no,pred_entity,pred_label,pred_start,pred_end,name_status


---------------------


### determine_principals

In [ ]:
#no_test

for index, row in demo_df.iterrows():
    entry_no = row['entry_no']
    entry_text = row['text']
    entities = ent_preds_df.loc[ent_preds_df['entry_no'] == entry_no]
    print(determine_principals(entry_text, entities, volume_metadata["type"]))
    if index > 25:
        break

Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None
Invalid number of principals.
None


### determine_event_date

In [ ]:
#no_test

for index, row in demo_df.iterrows():    
    entry_no = row['entry_no']
    entry_text = row['text']
    entities = ent_preds_df.loc[ent_preds_df['entry_no'] == entry_no]
    print(determine_event_date(entry_text, entities, volume_metadata["type"], volume_metadata))
    if index > 25:
        break       

None
None
None
None
Miércoles, día quatro de Diciembre de Mil Setecien tos Noventa y Tres. Yo
None
None
Jueves, día veinte y seis de Diciembre de Mil Setecientos Noventa y Tres. Yo
None
Lunes, día treze de Enero de Mil Setecientos Noventa y Quatro. Yo
Lunes, día tres de Febrero de Mil Setecientos Noventa y Quatro. Yo
Martes, día cinco de Marzo de Mil Setecientos Noventa y Quatro. Yo
Domingo, día diez y seis de Marzo de Mil Setecien tos Noventa y Quatro. Yo
None
None
Lunes, día quatorze de Abril de Mil Setecientos Noventa y Quatro. Yo
Sabado, día tres de Mayo de Mil Setecientos Noventa y Quatro. Yo
Sabado, día tres de Mayo de Mil Setecientos Noventa y Quatro. Yo
None
Miércoles  
Miércoles, día veinte y uno de Mayo de Mil Setecientos Noventa y Quatro. Yo
Miércoles, día veinte y uno de Mayo de Mil Setecientos Noventa y Quatro. Yo
Miércoles, día veinte y uno de Mayo de Mil Setecientos Noventa y Quatro. Yo
Miércoles, día veinte y uno de Mayo de Mil Setecientos Noventa y Quatro. Yo
Miércoles

The function currently fails to find a date for a substantial proportion (~20%) of entries because dates aren't being accurately extracted from the original. If this problem continues as the model improves and more entry data is incorporated into the sample, we'll need to add the post-processing capacity to bracket missing event dates by looking at entries on either side of the entry missing a date. Regardless, we will also need to add the post-processing capacity to convert these textual dates into numerical ones.

### determine_event_location

In [ ]:
#no_test

for index, row in demo_df.iterrows():    
    entry_no = row['entry_no']
    entry_text = row['text']
    entities = ent_preds_df.loc[ent_preds_df['entry_no'] == entry_no]
    print(determine_event_location(entry_text, entities, volume_metadata["type"], volume_metadata))
    if index > 25:
        break

Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the Diocese of St. Augustine
Archives of the 

### identify_cleric

In [ ]:
#no_test

for index, row in demo_df.iterrows():    
    entry_no = row['entry_no']
    entry_text = row['text']
    entities = ent_preds_df.loc[ent_preds_df['entry_no'] == entry_no]
    print(identify_cleric(entry_text, entities))
    if index > 25:
        break

Don Miguel o’Reilly
Don Miguel o’Reilly
Don Miguel o’Reilly
Don Miguel o’Reilly
Don Miguel o’Reilly
Don Miguel o’Reilly
Don Miguel o’Reilly
Don Miguel
Don Miguel o’Reilly
Don Miguel o’Reilly
Don Miguel o’Reilly
Don Miguel o’Reilly
Don Miguel o’Reilly
Don Miguel o’Reilly
Don Mi guel o’Reilly
Don Thomas Hafsett
Don Miguel O’Reilly
Don Thomas Hafsett
Don Miguel O’Reilly
Don Thomas Hafsett
Don Thomas Hafsett
Don Thomas Hafsett
Don Thomas Hafsett
Don Thomas Hafsett
Don Thomas Hafsett
Don Thomas Haf sett
Don Thomas Hafsett


In [ ]:
#no_test

from nbdev.export import notebook2script
notebook2script()

Converted 12-ssda-xml-parser.ipynb.
Converted 31-collate-xml-entities-spans.ipynb.
Converted 33-split-data.ipynb.
Converted 41-generic-framework-for-spacy-training.ipynb.
Converted 42-initial-model.ipynb.
Converted 51-data-preprocessing.ipynb.
Converted 52-unstructured-to-markup.ipynb.
Converted 53-markup-to-spatial-historian.ipynb.
Converted 61-prodigy-output-training-demo.ipynb.
Converted 62-full-model-application-demo.ipynb.
Converted 63-pt-model-training.ipynb.
Converted 64-es-model-training.ipynb.
Converted 71-relationship-builder.ipynb.
